In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import optuna
import pmdarima as pm
from statsmodels.tsa.arima.model import ARIMA

In [ ]:
df = pd.read_csv(r"C:\Users\Robyi\Documents\Data Science Dataset\elecproduction.csv")
df.dropna(inplace=True)
df.drop_duplicates(inplace=True)
df.head()

In [ ]:
df["date"] = pd.to_datetime(df["date"])
df.set_index("date", inplace=True)

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(df, label="Electricity Production", color="blue")
plt.xlabel("Time")
plt.ylabel("Production")
plt.title("Electricity Production Over Time")
plt.legend()
plt.show()

In [ ]:
model = pm.auto_arima(
    df, 
    seasonal=False,
    stepwise=True, 
    trace=True,
    error_action="ignore",
    suppress_warnings=True
)

In [ ]:
forecast, conf_int = model.predict(n_periods=1, return_conf_int=True)

In [ ]:
print(f"🔹 Prediksi Produksi Listrik Periode Berikutnya: {forecast[0]:.2f}")
print(f"🔹 Interval Kepercayaan 95%: ({conf_int[0][0]:.2f}, {conf_int[0][1]:.2f})")

In [ ]:
def objective(trial):
    p = trial.suggest_int("p", 0, 5)
    d = trial.suggest_int("d", 0, 2)
    q = trial.suggest_int("q", 0, 5)

    try:
        model = ARIMA(df, order=(p, d, q))
        model_fit = model.fit()
        aic = model_fit.aic
    except:
        aic = np.inf
    
    return aic

In [ ]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=30)

In [ ]:
best_params = study.best_params
print(f"🔹 Best ARIMA Order: (p={best_params['p']}, d={best_params['d']}, q={best_params['q']})")

In [ ]:
best_model = ARIMA(df, order=(best_params["p"], best_params["d"], best_params["q"]))
best_model_fit = best_model.fit()

In [ ]:
forecast_best, conf_int_best = best_model_fit.forecast(steps=1, alpha=0.05)

In [ ]:
print(f"🔹 Prediksi Terbaik Produksi Listrik: {forecast_best[0]:.2f}")
print(f"🔹 Interval Kepercayaan 95%: ({conf_int_best[0][0]:.2f}, {conf_int_best[0][1]:.2f})")